In [79]:
"""
------------------------------------------------------------------------------
About
In this data analysis, the proportion of mission calls (fire/ems) 
that reach the 10-minute response time is calculated for each admin area.

The sum of mission_count_ems, mission_count_fire and mission_count_technical_rescue 
was taken as the total sum of mission calls.
------------------------------------------------------------------------------
"""

'\n------------------------------------------------------------------------------\nAbout\nIn this data analysis, the proportion of mission calls (fire/ems) \nthat reach the 10-minute response time is calculated for each admin area.\n\nThe sum of mission_count_ems, mission_count_fire and mission_count_technical_rescue \nwas taken as the total sum of mission calls.\n------------------------------------------------------------------------------\n'

In [80]:
"""
------------------------------------------------------------------------------
Libraries
------------------------------------------------------------------------------
"""

import pandas as pd
import geopandas as gpd

import matplotlib.pyplot as plt

In [81]:
"""
------------------------------------------------------------------------------
Import data
------------------------------------------------------------------------------
"""

df = pd.read_csv("../data/raw/wyx48.csv", dtype={"planning_room_id":str})

df_pgr = pd.read_csv("../data/raw/PGR.csv", dtype={"PGR_ID": str})

df.head()

,planning_room_id,planning_room_name,mission_count_all,mission_count_ems,mission_count_ems_critical,mission_count_ems_critical_cpr,mission_count_fire,mission_count_technical_rescue,mission_count_ems_critical_timegoal_computed,mission_count_fire_timegoal_computed,...,response_time_fire_time_to_first_pump_std,response_time_fire_time_to_first_ladder_mean,response_time_fire_time_to_first_ladder_median,response_time_fire_time_to_first_ladder_std,response_time_fire_time_to_full_crew_mean,response_time_fire_time_to_full_crew_median,response_time_fire_time_to_full_crew_std,response_time_technical_rescue_mean,response_time_technical_rescue_median,response_time_technical_rescue_std
0,1100101,Stülerstraße,426,293,216,3,33,9,182,21,...,110.888486,693.518519,672.0,131.405529,744.571429,709.0,129.539018,921.625000,937.0,256.244826
1,1100102,Großer Tiergarten,1065,887,520,6,110,54,438,63,...,136.057777,732.686747,719.0,140.667999,843.904762,802.0,180.813459,804.857143,805.5,306.051133
2,1100103,Lützowstraße,697,534,373,6,36,29,331,10,...,153.350058,654.964286,662.0,108.650811,734.000000,734.0,105.222516,803.625000,794.5,318.006947
3,1100104,Körnerstraße,294,255,177,2,21,18,142,2,...,56.869303,736.555556,726.0,106.210237,696.500000,696.5,37.476659,972.062500,980.5,152.431610
4,1100205,Wilhelmstraße,581,500,332,5,51,30,276,23,...,91.957311,750.682927,740.0,113.773995,807.826087,800.0,96.747399,912.523810,855.0,329.384975


In [82]:

df_pgr.rename(columns={"PGR_ID": "pgr_id", "PGR_NAME":"pgr_name"}, inplace=True)

df_pgr.head()


,pgr_id,pgr_name,BEZ,STAND,GROESSE_M2
0,0930,Treptow-Köpenick 3,9,01.01.2021,4.068962e+07
1,0760,Marienfelde / Lichtenrade,7,01.01.2021,1.919230e+07
2,0920,Treptow-Köpenick 2,9,01.01.2021,1.920230e+07
3,0220,Kreuzberg Süd,2,01.01.2021,4.746238e+06
4,1260,Märkisches Viertel,12,01.01.2021,4.783336e+06


In [83]:
#add missing 0 in planning_room_id (LOR)
df["planning_room_id"] = df["planning_room_id"].str.zfill(8)
df.head()

,planning_room_id,planning_room_name,mission_count_all,mission_count_ems,mission_count_ems_critical,mission_count_ems_critical_cpr,mission_count_fire,mission_count_technical_rescue,mission_count_ems_critical_timegoal_computed,mission_count_fire_timegoal_computed,...,response_time_fire_time_to_first_pump_std,response_time_fire_time_to_first_ladder_mean,response_time_fire_time_to_first_ladder_median,response_time_fire_time_to_first_ladder_std,response_time_fire_time_to_full_crew_mean,response_time_fire_time_to_full_crew_median,response_time_fire_time_to_full_crew_std,response_time_technical_rescue_mean,response_time_technical_rescue_median,response_time_technical_rescue_std
0,01100101,Stülerstraße,426,293,216,3,33,9,182,21,...,110.888486,693.518519,672.0,131.405529,744.571429,709.0,129.539018,921.625000,937.0,256.244826
1,01100102,Großer Tiergarten,1065,887,520,6,110,54,438,63,...,136.057777,732.686747,719.0,140.667999,843.904762,802.0,180.813459,804.857143,805.5,306.051133
2,01100103,Lützowstraße,697,534,373,6,36,29,331,10,...,153.350058,654.964286,662.0,108.650811,734.000000,734.0,105.222516,803.625000,794.5,318.006947
3,01100104,Körnerstraße,294,255,177,2,21,18,142,2,...,56.869303,736.555556,726.0,106.210237,696.500000,696.5,37.476659,972.062500,980.5,152.431610
4,01100205,Wilhelmstraße,581,500,332,5,51,30,276,23,...,91.957311,750.682927,740.0,113.773995,807.826087,800.0,96.747399,912.523810,855.0,329.384975


In [84]:
pd.set_option("display.max_colwidth", None)
pd.set_option("display.width", 1000)

df["pgr_id"] = df["planning_room_id"].str[:4]

df.head()

,planning_room_id,planning_room_name,mission_count_all,mission_count_ems,mission_count_ems_critical,mission_count_ems_critical_cpr,mission_count_fire,mission_count_technical_rescue,mission_count_ems_critical_timegoal_computed,mission_count_fire_timegoal_computed,...,response_time_fire_time_to_first_ladder_mean,response_time_fire_time_to_first_ladder_median,response_time_fire_time_to_first_ladder_std,response_time_fire_time_to_full_crew_mean,response_time_fire_time_to_full_crew_median,response_time_fire_time_to_full_crew_std,response_time_technical_rescue_mean,response_time_technical_rescue_median,response_time_technical_rescue_std,pgr_id
0,01100101,Stülerstraße,426,293,216,3,33,9,182,21,...,693.518519,672.0,131.405529,744.571429,709.0,129.539018,921.625000,937.0,256.244826,0110
1,01100102,Großer Tiergarten,1065,887,520,6,110,54,438,63,...,732.686747,719.0,140.667999,843.904762,802.0,180.813459,804.857143,805.5,306.051133,0110
2,01100103,Lützowstraße,697,534,373,6,36,29,331,10,...,654.964286,662.0,108.650811,734.000000,734.0,105.222516,803.625000,794.5,318.006947,0110
3,01100104,Körnerstraße,294,255,177,2,21,18,142,2,...,736.555556,726.0,106.210237,696.500000,696.5,37.476659,972.062500,980.5,152.431610,0110
4,01100205,Wilhelmstraße,581,500,332,5,51,30,276,23,...,750.682927,740.0,113.773995,807.826087,800.0,96.747399,912.523810,855.0,329.384975,0110


In [85]:
#merge orgiginal data frame with admin area data frame
df_merged= pd.merge(df, df_pgr, on= "pgr_id", how="left")

In [86]:
"""
------------------------------------------------------------------------------
response_time_ems_critical_mean (in seconds)
response_time_ems_critical_std (in seconds)

response_time_fire_time_to_first_pump_mean (in seconds)
response_time_fire_time_to_first_pump_std (in seconds)

planning_room_name
------------------------------------------------------------------------------
"""
#subset
df_responseTime = df_merged[["pgr_id", "pgr_name", "response_time_ems_critical_mean", "response_time_ems_critical_std", "response_time_fire_time_to_first_pump_mean", "response_time_fire_time_to_first_pump_std"]]
df_responseTime.head()

,pgr_id,pgr_name,response_time_ems_critical_mean,response_time_ems_critical_std,response_time_fire_time_to_first_pump_mean,response_time_fire_time_to_first_pump_std
0,0110,Zentrum,593.758242,179.061417,556.222222,110.888486
1,0110,Zentrum,616.063927,209.523958,637.393258,136.057777
2,0110,Zentrum,589.604230,207.859285,588.366667,153.350058
3,0110,Zentrum,570.260563,196.087536,623.352941,56.869303
4,0110,Zentrum,611.612319,201.953546,643.435897,91.957311


In [88]:
#response time ems
df_responseTime["response_time_ems_critical_mean"] = (df_responseTime["response_time_ems_critical_mean"] /60).round(2)
df_responseTime["response_time_ems_critical_std"] = (df_responseTime["response_time_ems_critical_std"] /60).round(2)

#response time fire time to first pump
df_responseTime["response_time_fire_time_to_first_pump_mean"] = (df_responseTime["response_time_fire_time_to_first_pump_mean"] /60).round(2)
df_responseTime["response_time_fire_time_to_first_pump_std"] = (df_responseTime["response_time_fire_time_to_first_pump_std"] /60).round(2)

df_responseTime.head()

/var/folders/_f/022l68353h9c2pr_3t55ns1m0000gn/T/ipykernel_48269/2397338674.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_responseTime["response_time_ems_critical_mean"] = (df_responseTime["response_time_ems_critical_mean"] /60).round(2)
/var/folders/_f/022l68353h9c2pr_3t55ns1m0000gn/T/ipykernel_48269/2397338674.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_responseTime["response_time_ems_critical_std"] = (df_responseTime["response_time_ems_critical_std"] /60).round(2)
/var/folders/_f/022l

,pgr_id,pgr_name,response_time_ems_critical_mean,response_time_ems_critical_std,response_time_fire_time_to_first_pump_mean,response_time_fire_time_to_first_pump_std
0,0110,Zentrum,9.90,2.98,9.27,1.85
1,0110,Zentrum,10.27,3.49,10.62,2.27
2,0110,Zentrum,9.83,3.46,9.81,2.56
3,0110,Zentrum,9.50,3.27,10.39,0.95
4,0110,Zentrum,10.19,3.37,10.72,1.53


In [89]:
#group by admin area "Prognoseraum"
responseTimeEmsFire = df_responseTime.groupby(["pgr_id", "pgr_name"]).agg({"response_time_ems_critical_mean": "mean", 
                                                "response_time_ems_critical_std": "mean",
                                                "response_time_fire_time_to_first_pump_mean": "mean", 
                                                "response_time_fire_time_to_first_pump_std": "mean"
                                                }).reset_index()

responseTimeEmsFire

,pgr_id,pgr_name,response_time_ems_critical_mean,response_time_ems_critical_std,response_time_fire_time_to_first_pump_mean,response_time_fire_time_to_first_pump_std
0,0110,Zentrum,9.565000,3.223750,9.673125,1.981875
1,0120,Moabit,9.706154,3.130769,9.370000,2.480000
2,0130,Gesundbrunnen,10.094286,3.271429,9.698571,2.581429
3,0140,Wedding,9.651538,3.403077,9.246154,2.373077
4,0210,Kreuzberg Nord,9.321667,3.090000,9.045000,2.540000
5,0220,Kreuzberg Süd,9.681429,3.024286,9.394286,2.330000
6,0230,Kreuzberg Ost,8.978333,3.240000,9.206667,2.681667
7,0240,Friedrichshain West,9.146667,2.950000,9.340000,2.625000
8,0250,Friedrichshain Ost,9.785455,2.948182,9.914545,2.624545
9,0310,Buch,10.990000,3.533333,12.976667,2.423333


In [90]:
#sort descending
responseTimeEmsFire.sort_values(by= "response_time_ems_critical_mean", ascending=False, inplace=True)
responseTimeEmsFire.head()

,pgr_id,pgr_name,response_time_ems_critical_mean,response_time_ems_critical_std,response_time_fire_time_to_first_pump_mean,response_time_fire_time_to_first_pump_std
41,0930,Treptow-Köpenick 3,12.961111,3.774444,12.027778,2.045556
47,1040,Kaulsdorf/Mahlsdorf,12.495714,3.484286,11.028571,2.247143
10,0320,Nördliches Pankow,11.981429,3.687143,10.430000,2.246667
55,1240,Grüner Norden,11.860000,3.890000,12.617500,3.547500
34,0760,Marienfelde / Lichtenrade,11.756154,3.710769,11.455385,2.894545


In [91]:
#csv
responseTimeEmsFire.to_csv("../data/csv/responseTimeEMS.csv", index=False)